In [1]:
## Fun imports!
import sys
import os
import zipfile

## Plotting libraries
import pandas as pd   # Dataframes and Series are basically 2d and 1d tables 
import matplotlib.pyplot as plt
import seaborn as sns

## Library for creating vectors and linear algebra manipulation
## For Least Squares
from sklearn.datasets import make_regression
from matplotlib import pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
%matplotlib inline

## Used for reading files. ds100_utils is from Data 100
from ds100_utils import * 

## Imports from ds100_utils
import requests
from pathlib import Path
import time

import matplotlib.cm as cm

In [33]:
# Set up your sources and destination locations to download data from somewhere to somewhere on your computer

source_data_url = 'https://data.adb.org/media/6256/download'
target_file_name = 'ind-key-indicators-2020.xlsx'

# Change the force=False -> force=True in case you need to force redownload the data
dest_path = fetch_and_cache(
    data_url=source_data_url, 
    data_dir='.', 
    file=target_file_name, 
    force=False)

Using cached version that was downloaded (UTC): Mon Sep 28 11:51:11 2020


In [34]:
## The raw data
ind_key_indicators = pd.read_excel(target_file_name, skiprows=6, skip_footer=97)
ind_key_indicators.head()

,Unnamed: 0,Unnamed: 1,2000,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,NaN,POPULATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Total population as of 1 October (million),1019,1040,1056,1072,1089,1106,1122,1138,...,1186,1220.17,1236.22,1252.27,1268.31,1284.35,1299.43,1313.82,1328.21,1342.59
2,NaN,Population density (persons/km²),309,314,320,325,330,335,340,345,...,373,378,383,388,393,398,402.963,399.669,404.047,408.421
3,NaN,Population (% annual change),1.7982,2.06084,1.53846,1.51515,1.58582,1.56107,1.44665,1.42602,...,1.36752,1.32882,1.29878,1.27043,1.24114,1.2,1.2,1.10671,1.09536,1.08266
4,NaN,Urban population (% of total population),27.7,27.9,28.1,28.3,28.5,28.8,29,29.2,...,29.9,31.34,31.68,32.01,32.34,32.67,33,33.32,33.65,33.97


## Individual Parsing and Cleaning

By looking at the excel sheet directly, we notice that the column `Unnamed: 0`  is unnecessary. so we can drop it.

In [36]:
## Cleaning
ind_key_indicators.columns
ind_key_indicators = ind_key_indicators.drop('Unnamed: 0', axis=1)

We can continue cleaning it. We notice that there are also categories along with subcategoeis for each of the data. From this specific format, it may be easiest to have separate dataframe for each category, where each row is a subcategory and each year is the corresponding year.

In [37]:
population = ind_key_indicators.iloc[1:5]
population

,Unnamed: 1,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Total population as of 1 October (million),1019,1040,1056,1072,1089,1106,1122,1138,1154,...,1186,1220.17,1236.22,1252.27,1268.31,1284.35,1299.43,1313.82,1328.21,1342.59
2,Population density (persons/km²),309,314,320,325,330,335,340,345,350,...,373,378,383,388,393,398,402.963,399.669,404.047,408.421
3,Population (% annual change),1.7982,2.06084,1.53846,1.51515,1.58582,1.56107,1.44665,1.42602,1.40596,...,1.36752,1.32882,1.29878,1.27043,1.24114,1.2,1.2,1.10671,1.09536,1.08266
4,Urban population (% of total population),27.7,27.9,28.1,28.3,28.5,28.8,29,29.2,29.3,...,29.9,31.34,31.68,32.01,32.34,32.67,33,33.32,33.65,33.97


In [32]:
labor_force = ind_key_indicators.iloc[7:26]
labor_force

,Unnamed: 1,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
7,Employed,336.61,…,…,…,…,368.875,…,…,…,...,…,…,…,…,…,…,…,…,…,…
8,"Agriculture, forestry, and fishing",201.781,…,…,…,…,207.076,…,…,…,...,…,…,…,…,…,…,…,…,…,…
9,Mining and quarrying,54.8932,…,…,…,…,69.3901,…,…,…,...,…,…,…,…,…,…,…,…,…,…
10,Manufacturing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"Electricity, gas, steam, and air-conditio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Wholesale and retail trade; repair of mot...,79.9361,…,…,…,…,92.4085,…,…,…,...,…,…,…,…,…,…,…,…,…,…
14,Accommodation and food service activities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Transportation and storage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Information and communication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
